# Milestone Project 1: Food Vision Big

## Check GPU

* A100
* T4
* V100


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-33665966-c94f-1edc-ca44-378c99ff845f)


In [1]:
# Get helper functions (Download helper functions script from JonasHagstrom79/Tensorflow(raw))
!wget https://raw.githubusercontent.com/JonasHagstrom79/Tensorflow/main/helper.py



--2024-01-29 20:07:20--  https://raw.githubusercontent.com/JonasHagstrom79/Tensorflow/main/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13380 (13K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]  13.07K  --.-KB/s    in 0s      

2024-01-29 20:07:20 (129 MB/s) - ‘helper.py’ saved [13380/13380]



In [2]:
# Import series of helper functions
from helper import create_tensorboard_callback, plot_loss_curves, compare_historys